# Data Transformation

Chama is a relatively new, modern company, with an IOS App and Android App. To grow our company we must make a special effort in collecting all information available around the App and make it available for everyone in the company. Some information about App usage is generated by our App and our backend API. Other sources of information, like the Google Play Store, can provide very useful insights on App performance and user rating. Chama is very data-oriented and, most decisions are made based on appropriate metrics, therefore, data quality is a must and a concern of everyone involved.

Some event data come as json files and need some transformation to be structured as tables.

***
**CuratedOfferOptions.csv**:
***
**CurationProvider**: in quotes<br>
**OfferId**: in quotes<br>
**DealerId**: in quotes<br>
**UniqueOptionId**: in quotes<br>
**OptionId**: in quotes<br>
**IsMobileDealer**: without quotes<br>
**IsOpen**: without quotes<br>
**Eta**: in quotes<br>
**ChamaScore**: without quotes<br>
**ProductBrand**: in quotes<br>
**IsWinner**: without quotes<br>
**MinimumPrice**: without quotes
**MaximumPrice**: without quotes<br>
**DynamicPrice**: without quotes<br>
**FinalPrice**: without quotes<br>
**DefeatPrimaryReason**: in quotes<br>
**DefeatReasons**: in quotes<br>
**EnqueuedTimeSP**: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)<br>

***
**DynamicPriceOption.csv**:
***
**Provider**: in quotes<br>
**OfferId**: in quotes<br>
**UniqueOptionId**: in quotes<br>
**BestPrice**: without quotes<br>
**EnqueuedTimeSP**: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)<br>

***
**DynamicPriceRange.csv**:
***
**Provider**: in quotes<br>
**OfferId**: in quotes<br>
**MinGlobal**: without quotes<br>
**MinRecommended**: without quotes<br>
**MaxRecommended**: without quotes<br>
**DifferenceMinRecommendMinTheory**: without quotes<br>
**EnqueuedTimeSP**: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)<br>

# Importing required libraries

In [150]:
import numpy as np
import pandas as pd
import os

In [151]:
for dirpath, dirnames, filenames in os.walk(top='./'):
    print(f'{dirpath}--->{dirnames}--->{filenames}')

./--->['datasets', '.ipynb_checkpoints']--->['.DS_Store', 'Chama.ipynb']
./datasets--->[]--->['case.json']
./.ipynb_checkpoints--->[]--->['Chama-checkpoint.ipynb']


# Basic Data Exploration

In [152]:
raw_data = pd.read_json('./datasets/case.json')

In [153]:
raw_data.shape

(37, 3)

In [154]:
raw_data.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 08:04:08 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 11:43:23 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 09:04:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,2021-08-25 05:02:55 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
4,2021-09-05 08:03:28 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


In [155]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   EnqueuedTimeUtc  37 non-null     object
 1   EventName        37 non-null     object
 2   Payload          37 non-null     object
dtypes: object(3)
memory usage: 1020.0+ bytes


# Processing EnqueuedTimeUtc column

In [156]:
raw_data['EnqueuedTimeUtc'] = pd.to_datetime(raw_data['EnqueuedTimeUtc'])

In [157]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   EnqueuedTimeUtc  37 non-null     datetime64[ns, UTC]
 1   EventName        37 non-null     object             
 2   Payload          37 non-null     object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 1020.0+ bytes


In [158]:
raw_data['EnqueuedTimeUtc'].dt.tz

<UTC>

In [159]:
import zoneinfo

In [160]:
timezones = pd.DataFrame({'Timezones':list(zoneinfo.available_timezones())})

In [161]:
timezones[timezones.Timezones.str.contains('Brazil')]

,Timezones
143,Brazil/West
298,Brazil/East
313,Brazil/DeNoronha
471,Brazil/Acre


In [162]:
# converting to UTC-3 timezone
raw_data['EnqueuedTimeUtc'] = raw_data['EnqueuedTimeUtc'].dt.tz_convert(tz='Brazil/East')

In [163]:
raw_data = raw_data.rename(columns={'EnqueuedTimeUtc':'Enqueued_Time'})

In [164]:
raw_data.Enqueued_Time.dt.tz

<DstTzInfo 'Brazil/East' LMT-1 day, 20:54:00 STD>

# Separate data based on EventName

In [165]:
raw_data['EventName'].value_counts()

EventName
DynamicPrice_Result    32
CurateOffer_Result      5
Name: count, dtype: int64

**observations**<br>
1. EventName Indicates which csv file the respective record belongs to
2. Upon close inspection on the provider in the payload column, DynamicPrice_Result is further split into 2 categories <br>
       - DynamicPriceRange<br>
       - DynamicPricePerOption<br>

In [166]:
DynamicPrice = raw_data.query('EventName == "DynamicPrice_Result"')
CuratedOfferOption = raw_data.query('EventName == "CurateOffer_Result"').reset_index().drop(columns='index')

In [167]:
DynamicPriceOption = DynamicPrice[DynamicPrice.Payload.str.contains('ApplyDynamicPricePerOption')].reset_index().drop(columns='index')

In [168]:
DynamicPriceRange = DynamicPrice[DynamicPrice.Payload.str.contains('ApplyDynamicPriceRange')].reset_index().drop(columns='index')

# More Processing

In [169]:
DynamicPriceOption.Payload[0]

'{"provider":"ApplyDynamicPricePerOption","offerId":"56e0702c-0218-4626-8d3d-ae9d54b4503b","algorithmOutput":[{"uniqueOptionId":"b0e296a9-0590-f0e0-8211-243a2ededb12","bestPrice":92.45},{"uniqueOptionId":"d6562c24-0b37-5fb4-8275-65b7b8b47b87","bestPrice":92.45},{"uniqueOptionId":"8d0f9262-f543-d0c8-a869-33985ae3ecda","bestPrice":92.45},{"uniqueOptionId":"151e59ac-761a-96f5-d2b9-882037a9fd28","bestPrice":94.6},{"uniqueOptionId":"3cd346f4-d297-7568-2e50-d43a8e2fd0a9","bestPrice":94.6},{"uniqueOptionId":"b7a7b6d1-4dae-7392-5aaf-f3369c29db1d","bestPrice":93.0},{"uniqueOptionId":"577e4bbd-f49d-ac23-56a6-e70072a05229","bestPrice":93.0},{"uniqueOptionId":"f9b876ab-2590-952f-d69d-5b352ec251f3","bestPrice":91.35}]}'

In [170]:
DynamicPriceRange.Payload[0]

'{"provider":"ApplyDynamicPriceRange","offerId":"a6611d55-9624-4381-8cdd-323ee3689241","algorithmOutput":{"min_global":85.0,"min_recommended":87.2,"max_recommended":97.65,"differenceMinRecommendMinTheory":2.2}}'

**observations**<br>
    - values in Payload cloumn are str type, it would be better to convert them to type dict

In [171]:
CuratedOfferOption.Payload[0]

'[{"curationProvider":"ByPrice","offerId":"149f0e53-ff85-425f-a01a-8710f06704ea","dealerId":"6517","options":[{"uniqueOptionId":"b0e296a9-0590-f0e0-8211-243a2ededb12","optionId":"6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2","isMobileDealer":true,"isOpen":true,"eta":"1:00","chamaScore":8.0,"productBrand":"ULTRAGAZ","isWinner":true,"minimumPrice":90.00,"maximumPrice":180.00,"dynamicPrice":91.90,"finalPrice":91.90},{"uniqueOptionId":"d6562c24-0b37-5fb4-8275-65b7b8b47b87","optionId":"6517 || 6517","isMobileDealer":false,"isOpen":false,"eta":"0:01","chamaScore":8.0,"productBrand":"ULTRAGAZ","isWinner":false,"minimumPrice":90.00,"maximumPrice":180.00,"dynamicPrice":91.90,"finalPrice":91.90,"defeatPrimaryReason":"Closed","defeatReasons":["Closed","HasDriverInOffer"]}]},{"curationProvider":"ByPrice","offerId":"149f0e53-ff85-425f-a01a-8710f06704ea","dealerId":"9047","options":[{"uniqueOptionId":"8d0f9262-f543-d0c8-a869-33985ae3ecda","optionId":"9047 || 9047 || ULTRAGAZ","isMobileDealer":false,

**observations**<br>
    - values in Payload cloumn are str type, it would be better to convert them to type list<br>

In [172]:
import json

In [173]:
def fromJson(Payload):
    return json.loads(Payload)

In [174]:
DynamicPriceOption.loc[:,'Payload'] = DynamicPriceOption.Payload.apply(fromJson)
DynamicPriceRange.loc[:,'Payload'] = DynamicPriceRange.Payload.apply(fromJson)

In [175]:
CuratedOfferOption.loc[:,'Payload'] = CuratedOfferOption.Payload.apply(fromJson)

# Data Extraction Classes

In [176]:
class TransformDPO:

    def __init__(self):
        pass

    def transform(self,df):
        data = []
        
        for _,row in df.iterrows():
            Provider = f"\"{row.Payload.get('provider')}\""
            OfferId = f"\"{row.Payload.get('offerId')}\""
            EnqueuedTimeSP = row['Enqueued_Time'].strftime('%d/%m/%Y')
            for option in row.Payload['algorithmOutput']:
             UniqueOptionId = f"\"{option.get('uniqueOptionId')}\""
             BestPrice = option.get('bestPrice')
             data.append([Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSP])

        return pd.DataFrame(data,columns=['Provider','OfferId','UniqueOptionId','BestPrice','EnqueuedTimeSP'])

In [177]:
dynamicPriceOption_df = TransformDPO().transform(DynamicPriceOption)

In [178]:
dynamicPriceOption_df.to_csv('./DynamicPriceOption.csv',index=False)

In [179]:
class TransformDPR:

    def __init__(self):
        pass

    def transform(self,df):
        data = []
        
        for _,row in df.iterrows():
            Provider = f"\"{row.Payload.get('provider')}\""
            OfferId = f"\"{row.Payload.get('offerId')}\""
            MinGlobal = row.Payload['algorithmOutput']['min_global']
            MinRecommended = row.Payload['algorithmOutput']['min_recommended']
            MaxRecommended = row.Payload['algorithmOutput']['max_recommended']
            DifferenceMinRecommendMinTheory = row.Payload['algorithmOutput']['differenceMinRecommendMinTheory']
            EnqueuedTimeSP = row['Enqueued_Time'].strftime('%d/%m/%Y')

            data.append([Provider,OfferId,MinGlobal,MinRecommended,MaxRecommended,
                         DifferenceMinRecommendMinTheory,EnqueuedTimeSP])

        return pd.DataFrame(data,columns=['Provider','OfferId','MinGlobal',
                                          'MinRecommended','MaxRecommended','DifferenceMinRecommendMinTheory',
                                          'EnqueuedTimeSP'])

In [180]:
dynamicPriceRange_df = TransformDPR().transform(DynamicPriceRange)

In [181]:
dynamicPriceRange_df.to_csv('./DynamicPriceRange.csv',index=False)

In [187]:
class TransformCOO:

    def __init__(self):
        pass

    def transform(self,df):
        data = []
        
        for _,row in df.iterrows():
            EnqueuedTimeSP = row['Enqueued_Time'].strftime('%d/%m/%Y')

            for item in row.Payload:
                CurationProvider = f"\"{item.get('curationProvider')}\""
                OfferId = f"\"{item.get('offerId')}\""
                DealerId = f"\"{item.get('dealerId')}\""

                for option in item.get('options'):
                    UniqueOptionId = f"\"{option.get('uniqueOptionId')}\""
                    OptionId = f"\"{option.get('optionId')}\""
                    IsMobileDealer = option.get('isMobileDealer')
                    IsOpen = option.get('isOpen')
                    Eta = f"\"{option.get('eta')}\""
                    ChamaScore = option.get('chamaScore')
                    ProductBrand = f"\"{option.get('productBrand')}\""
                    IsWinner = option.get('isWinner')
                    MinimumPrice = option.get('minimumPrice')
                    MaximumPrice = option.get('maximumPrice')
                    DynamicPrice = option.get('dynamicPrice')
                    FinalPrice = option.get('finalPrice')
                    DefeatPrimaryReason = f"\"{option.get('defeatPrimaryReason','-')}\""
                    DefeatReasons = f"\"{option.get('defeatReasons','-')}\""
                    
                    data.append([CurationProvider,OfferId,DealerId,
                                 UniqueOptionId,OptionId,IsMobileDealer,IsOpen,
                                 Eta,ChamaScore,ProductBrand,IsWinner,
                                 MinimumPrice,MaximumPrice,DynamicPrice,
                                 FinalPrice,DefeatPrimaryReason,DefeatReasons])

        return pd.DataFrame(data,columns=['CurationProvider','OfferId','DealerId',
                                          'UniqueOptionId','OptionId','IsMobileDealer','IsOpen',
                                          'Eta','ChamaScore','ProductBrand','IsWinner',
                                          'MinimumPrice','MaximumPrice','DynamicPrice',
                                          'FinalPrice','DefeatPrimaryReason','DefeatReasons'])

In [188]:
curatedOfferOptions_df = TransformCOO().transform(CuratedOfferOption)

In [189]:
curatedOfferOptions_df.to_csv('./CuratedOfferOptions.csv',index=False)